# Covid and Influenza - sick and death cases comparison

**Part 1 - Cleaning / Transforming data**

In here I am going to check how the influenza detected cases has been changed after covid19 pandemic

**Sources:**
- Covid19: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data


- Influenza: https://flunewseurope.org/VirusCharacteristics

**Covid19 source - Terms of Use:**
1.	This data set is licensed under the Creative Commons Attribution 4.0 International (CC BY 4.0) by the Johns Hopkins University on behalf of its Center for Systems Science in Engineering. Copyright Johns Hopkins University 2020.
2.	Attribute the data as the "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University" or "JHU CSSE COVID-19 Data" for short, and the url: https://github.com/CSSEGISandData/COVID-19.
3.	For publications that use the data, please cite the following publication: "Dong E, Du H, Gardner L. An interactive web-based dashboard to track COVID-19 in real time. Lancet Inf Dis. 20(5):533-534. doi: 10.1016/S1473-3099(20)30120-1"

In [ ]:
import pandas as pd
import os
from pathlib import Path


from bs4 import BeautifulSoup
import requests
import re 
import io
import numpy as np

# A) Influenza
Let's start from Influenza virus detections by type in Europe

## A1. Extract Data
All data have been downloaded from [here](https://flunewseurope.org/VirusCharacteristics). I have filtered by all countries and all dates from the list. I transfered it to my github repository just after I have downloaded it as .xlsx from the website page

In [ ]:
url_flu = "https://raw.githubusercontent.com/mborycki/Covid_Influenza_Comparison/main/Influenza_virus_detections_in_Europe.csv"

In [ ]:
flu_detected = pd.read_csv(url_flu)

In [ ]:
flu_detected.head()

## A2. Transform / Clean Data

In [ ]:
flu_detected['Region'].unique()

In [ ]:
flu_detected[(flu_detected.Country=='Poland')&(flu_detected.Week=='2015-W40')]

In [ ]:
len(flu_detected[(flu_detected.Country=='Poland')&(flu_detected.Region=='EU/EEA')])

In [ ]:
len(flu_detected[(flu_detected.Country=='Poland')&(flu_detected.Region=='WHO Europe')])

In [ ]:
flu_detected[(flu_detected.Country=='Poland')&(flu_detected.Region=='WHO Europe')].iloc[:,5:].values.sum()

In [ ]:
flu_detected[(flu_detected.Country=='Poland')&(flu_detected.Region=='EU/EEA')].iloc[:,5:].values.sum()

In [ ]:
len(flu_detected[flu_detected.Region=='EU/EEA'])

In [ ]:
print(f"In EEA we have {len(flu_detected[flu_detected.Region=='EU/EEA'])} records while in WHO Europe we have"
      f" {len(flu_detected[flu_detected.Region=='WHO Europe'])} records")

In [ ]:
flu_detected = flu_detected[flu_detected.Region == 'WHO Europe']

In [ ]:
flu_detected.info()

**First conclusion:**
- Change coumn name: YearWeek - Week
- What is "Surveillance System Type" and do we need it?
- Do we need "Season" and "Region" columns? If not then remove them
- Create total cases for flu
- Unpivot the table
- Split YearWeek column into 2 separate

In [ ]:
flu_detected.rename(columns={'Week':'YearWeek'},inplace=True)

In [ ]:
flu_detected.columns

In [ ]:
flu_detected['Surveillance System Type'].unique()

In [ ]:
flu_detected['Season'].unique()

In [ ]:
flu_detected['YearWeek'].unique()[:5]

So, I do not need columns "Surveillance System Type" (_one unique value: 'Non-sentinel'_), "Season" (_Week is more precise_) and "Region" (_only 'WHO Europe'_)

In [ ]:
# this column is just in one table. So I did not have it in the above function
flu_detected = flu_detected.drop(['Season','Region','Surveillance System Type'],axis=1) 

In [ ]:
flu_detected.columns

In [ ]:
flu_detected.iloc[:,2:].head(2) # we will summarize it from the third column

In [ ]:
# Create a total flu detected cases
flu_detected['Total Detected Cases'] = flu_detected.iloc[:,2:].sum(axis=1).values
flu_detected.head()

In [ ]:
flu_detected[(flu_detected['Total Detected Cases']>0)&(flu_detected.YearWeek=='2021-W01')]

In [ ]:
flu_detected[(flu_detected.Country=='Poland')&(flu_detected.YearWeek=='2016-W04')] # looks ok

In [ ]:
# Unpivot table 
def FluPivot(df,colname):
    x = pd.melt(df, id_vars=['Country', 'YearWeek'], var_name='Flu Type', value_name='Cases').sort_values(['YearWeek','Country'])
    x[['Year', 'Week']] = x['YearWeek'].str.split('-', n=1, expand=True)
    x = x.drop(["YearWeek"],axis=1)
    x.rename(columns={'Cases':colname},inplace=True)
    return x

In [ ]:
flu_detected2 = FluPivot(flu_detected,'Detected_Cases')
flu_detected2.head()

In [ ]:
flu_detected2['Flu Type'].unique()

In [ ]:
flu_detected2.info()

In [ ]:
flu_detected2[(flu_detected2.Country=='Poland')&(flu_detected2.Year=='2016')\
              &(flu_detected2['Flu Type']=='Total Detected Cases')]['Detected_Cases'].sum() # looks ok

In [ ]:
flu_detected2[(flu_detected2.Country=='Poland')&(flu_detected2.Year=='2016')]

In [ ]:
# Rename Flu Type names
original_type_names = ['A not subtyped', 'A(H1)pdm09', 'A(H3)',
       'B lineage not determined', 'B/Vic', 'B/Yam']

new_type_names = ['A', 'A(H1)', 'A(H3)','B', 'B/Vic', 'B/Yam']

for o, n in zip(original_type_names,new_type_names):
    flu_detected2.loc[(flu_detected2['Flu Type'] == o),'Flu Type']=n

In [ ]:
flu_detected2.head()

In [ ]:
flu_detected2['Flu Type'].unique()

In [ ]:
# Amount od total cases
flu_detected2[(flu_detected2['Flu Type']=='Total Detected Cases')].sort_values(['Year']).groupby(['Flu Type','Year'])\
['Detected_Cases'].sum().reset_index()

In [ ]:
# Amount of records
flu_detected2[(flu_detected2['Flu Type']=='Total Detected Cases')].groupby('Year')['Flu Type'].count()

In [ ]:
years = flu_detected2.Year.unique()
for year in years:
    print(f'In {year} we have {len(flu_detected2[flu_detected2.Year==year].Week.unique())} weeks')

In [ ]:
# remove unnecesary weeks
years_list = ['2016','2017','2018','2019','2020', '2021'] # we keep 2021 as covid data are available for this year
flu_detected2 = flu_detected2[flu_detected2['Year'].isin(years_list)]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df1 = flu_detected2[flu_detected2['Flu Type']!='Total Detected Cases'].groupby(['Flu Type', 'Year'])['Detected_Cases'].sum().reset_index()
df2 = df1.pivot(index="Year", columns="Flu Type", values="Detected_Cases").reset_index().set_index('Year')

fig, ax1 = plt.subplots()
plt.rcParams["figure.figsize"] = (25,15)
plt.xticks(fontsize=16, rotation=45)
plt.grid(color='grey', linestyle = '--', linewidth = 0.5)

width = 0.8
bottom = 0

for i in df2.columns:
    plt.bar(df2.index, df2[i], width=width, bottom=bottom)
    bottom += df2[i]

plt.title(f"Influenza Cases per year in Europe", fontsize=28)
plt.xlabel('Years', fontsize=24)
plt.ylabel("Detected Flu Cases", color='black', fontsize=24)
plt.tick_params(axis='y', labelcolor='black', labelsize=16) 
plt.legend(df2.columns, fontsize=16)
plt.tight_layout()

In [ ]:
flu_detected2[(flu_detected2.Country=='Poland')&(flu_detected2['Flu Type']=='Total Detected Cases')].sort_values(['Year'])\
.groupby(['Flu Type','Year'])['Detected_Cases'].sum().reset_index()

---
Remove all Flu Types except the totals

In [ ]:
df_flu = flu_detected2[(flu_detected2['Flu Type']=='Total Detected Cases')].sort_values(['Year'])\
.groupby(['Country','Year', 'Week'])['Detected_Cases'].sum().reset_index()

In [ ]:
df_flu.head(2)

In [ ]:
df_flu.groupby('Year')['Detected_Cases'].sum().reset_index()

In [ ]:
# total for all countries
xs = df_flu.groupby('Year')['Detected_Cases'].sum().reset_index()['Year']
ys = df_flu.groupby('Year')['Detected_Cases'].sum().reset_index().Detected_Cases.values

plt.rcParams["figure.figsize"] = (15,10)
plt.plot(xs,ys,'bo-')
plt.title(f'Total influenza cases for all countries', fontsize=24)

for x,y in zip(xs,ys):

    #label = "{:.0f}".format(y)
    label = f'{y:,}'
    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)

plt.show()

In [ ]:
# total for chosen country
country = 'Poland'
xs = df_flu[df_flu.Country==country].groupby('Year')['Detected_Cases'].sum().reset_index()['Year']
ys = df_flu[df_flu.Country==country].groupby('Year')['Detected_Cases'].sum().reset_index().Detected_Cases.values

plt.rcParams["figure.figsize"] = (15,10)
plt.plot(xs,ys,'bo-')
plt.title(f'Total influenza cases in {country}', fontsize=24)

for x,y in zip(xs,ys):

    #label = "{:.0f}".format(y)
    label = f'{y:,}'
    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)

plt.show()

In [ ]:
df_flu['Week'] = df_flu['Week'].map(lambda x: x.lstrip('W'))
df_flu['Week'] = df_flu['Week'].astype('int')
df_flu['Year'] = df_flu['Year'].astype('int')

In [ ]:
df_flu.info()

In [ ]:
df_flu.sort_values(['Year','Week']).head(10)

In [ ]:
df_flu[(df_flu.Country=='Poland')].sort_values(['Year'])\
.groupby(['Year'])['Detected_Cases'].sum().reset_index()

The table with influenza data seems to be clean and ready to use. Now, let's check the covid table

# B) Covid19

## B1. Scrape Data
Please find all details arond the report in the [Daily reports (csse_covid_19_daily_reports)](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports)
This folder contains daily case reports. All timestamps are in UTC (GMT+0).

In the report, there is many different columns but we will use just some of them:

---
**Field description**
* <b>Country_Region</b>: Country, region or sovereignty name. The names of locations included on the Website correspond with the official designations used by the U.S. Department of State.
* <b>Last Update</b>: MM/DD/YYYY HH:mm:ss  (24 hour format, in UTC).
* <b>Lat</b> and <b>Long_</b>: Dot locations on the dashboard. All points (except for Australia) shown on the map are based on geographic centroids, and are not representative of a specific address, building or any location at a spatial scale finer than a province/state. Australian dots are located at the centroid of the largest city in each state.
* <b>Confirmed</b>: Counts include confirmed and probable (where reported).
* <b>Deaths</b>: Counts include confirmed and probable (where reported).
* <b>Recovered</b>: Recovered cases are estimates based on local media reports, and state and local reporting when available, and therefore may be substantially lower than the true number. US state-level recovered cases are from [COVID Tracking Project](https://covidtracking.com/).
* <b>File_Name</b>: It is a column created by me contain the name of the .csv file with data (in a date format)

In [ ]:
url = "https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports"
r = requests.get(url)

In [ ]:
html_doc = r.text
soup = BeautifulSoup(html_doc)
a_tags = soup.find_all('a')

# Store a list of urls ending in .csv: urls => list
urls = ['https://raw.githubusercontent.com'+re.sub('/blob', '', link.get('href'))
        for link in a_tags if '.csv' in link.get('href')]

# Store a list of Data Frame names to be assigned to the list: df_list_names => list
df_list_names = [url.split('.csv')[0].split('/')[url.count('/')] for url in urls]

In [ ]:
urls[:5]

In [ ]:
df_list_names[:5]

In [ ]:
len(urls)

In [ ]:
comments = pd.DataFrame(columns=['File_Name','Added','Not_Added'])
comments.head()

In [ ]:
cols = ['Country_Region','Last_Update','Lat','Long_','Confirmed','Deaths','Recovered','File_Name']

covid_table = pd.DataFrame(columns = cols)
covid_table.head()

In [ ]:
def Add_Comment(url_name, is_ok, is_not_ok):
    data = [[url_name,is_ok,is_not_ok]]
    comment_note = pd.DataFrame(data, columns = ['File_Name','Added','Not_Added'])
    return comment_note

In [ ]:
# some tables have a bit different column names, if so then I keep only the below names
col_names1 = ['Country_Region','Last_Update','Lat','Long_','Confirmed','Deaths','Recovered']
col_names2 = ['Country/Region','Last Update','Latitude','Longitude','Confirmed','Deaths','Recovered']
col_names3 = ['Country/Region','Last Update','Confirmed','Deaths','Recovered']

In [ ]:
url

In [ ]:
for count, url in enumerate(urls):
    download = requests.get(url).content
    # Reading the downloaded content and turning it into a pandas dataframe
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    if (df.shape[1] == 14) | (df.shape[1] == 12):
        df = df[col_names1]
    elif df.shape[1] == 8:
        df = df[col_names2]
    elif df.shape[1] == 6:
        df = df[col_names3]
        df['Lat'] = 0
        df['Long_'] = 0
        df = df[['Country/Region','Last Update','Lat','Long_','Confirmed','Deaths','Recovered']]
    else:
        print(f'We have {df.shape[1]} columns in {url} file')
    
    df['File_Name'] = df_list_names[count]        
    try:
        df.columns = cols # renaming the columns
        covid_table = covid_table.append(df, ignore_index=True)
        comment = Add_Comment(df_list_names[count], 1, 0)
        comments = comments.append(comment, ignore_index=True)
    except:
        comment = Add_Comment(df_list_names[count], 0, 1)
        comments = comments.append(comment, ignore_index=True)
        pass

In [ ]:
df.head()

In [ ]:
covid_table.head()

In [ ]:
covid_table.info()

In [ ]:
# checking if there is any file I did not download
comments[['Added','Not_Added']].sum().plot(kind='bar')

## B2. Transform / Clean Data

In [ ]:
df_covid = covid_table.copy()

In [ ]:
# changing format for 2 columns with dates
df_covid['File_Name'] = pd.to_datetime(df_covid['File_Name']).dt.date
df_covid['Last_Update'] = pd.to_datetime(df_covid['Last_Update']).dt.date

In [ ]:
df_covid.fillna({'Deaths':0,'Confirmed':0,'Recovered':0},inplace=True)

In [ ]:
covid_cases_list = ['Confirmed','Deaths','Recovered']
for case in covid_cases_list:
    df_covid[case] = df_covid[case].astype(float)

In [ ]:
df_covid.info()

In [ ]:
df_covid.head()

In [ ]:
df_covid = df_covid.groupby(['Country_Region','Last_Update']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()

In [ ]:
df_covid.head()

In [ ]:
# getting week/year as numbers
week_no = []
year_no = []
for value in df_covid['Last_Update']:
    week_no.append(value.isocalendar()[1])
    year_no.append(value.isocalendar()[0])

df_covid['Week'] = week_no
df_covid['Year'] = year_no

In [ ]:
df_covid[(df_covid.Country_Region=='Poland')&(df_covid.Week==20)&(df_covid.Year==2021)]

In [ ]:
df_covid = df_covid.groupby(['Country_Region','Week','Year'], sort=False).agg({'Confirmed':'max','Deaths':'max','Recovered':'max'}).reset_index()

In [ ]:
df_covid[(df_covid.Country_Region=='Poland')&(df_covid.Week==20)&(df_covid.Year==2021)]

In [ ]:
df_covid[(df_covid.Country_Region=='Poland')]['Confirmed'].max()

In [ ]:
df_covid.Year.unique()

Now the table look quite well. Now I am going to combine the table with Influenza table

---

## Combining Tables
We need to keep all outputs in one table

In [ ]:
df_covid.rename(columns={'Country_Region':'Country'},inplace=True)

First tables correlations need to be check

In [ ]:
df_covid.info()

In [ ]:
df_flu.info()

In [ ]:
df_covid.head()

In [ ]:
df_flu.head()

The correlations seems to be ok. Now, I have concerns whether the countries have the same names in both tables

In [ ]:
flu_countries = df_flu[['Country']].drop_duplicates()
flu_countries['flu'] = 1
cov_countries = df_covid[['Country']].drop_duplicates()
cov_countries['cov'] = 1

In [ ]:
countries = pd.merge(flu_countries,cov_countries,on='Country',how='outer')

In [ ]:
countries[countries['cov'].isnull()]

We can see countries names which are different in both tables. Of course the names need to be the same in both tables

In [ ]:
missing_countries = ['Herzego','Koso','Mold','Mace','Turkm','Kingdom']
print('Covid:')
for c in missing_countries:
    print(df_covid[df_covid.Country.str.contains(c)].Country.unique()) #flu_df2
    
print('\nInfluenza:')
for c in missing_countries:
    print(df_flu[df_flu.Country.str.contains(c)].Country.unique()) #flu_df2

In [ ]:
missing_countries = ['Herzego','Koso','Mold','Mace','Kingdom', 'Rus']
new_countries = ['Bosnia and Herzegovina','Kosovo','Moldova','Macedonia','United Kingdom', 'Russia'] 

for old,new in zip(missing_countries,new_countries):
    df_covid.loc[df_covid.Country.str.contains(old), 'Country'] = new
    df_flu.loc[df_flu.Country.str.contains(old), 'Country'] = new

In [ ]:
print('Covid:')
for c in missing_countries:
    print(df_covid[df_covid.Country.str.contains(c)].Country.unique()) #flu_df2
    
print('\nInfluenza:')
for c in missing_countries:
    print(df_flu[df_flu.Country.str.contains(c)].Country.unique()) #flu_df2

Currently, the coutries names are the same (Turkmenistan is missing in Covid table)

In [ ]:
df_covid.head()

In [ ]:
df_covid.shape

In [ ]:
df_flu.rename(columns={'Detected_Cases':'Detected_FluCases'},inplace=True)

In [ ]:
df_flu.head()

In [ ]:
df_flu.shape

In [ ]:
# df_flu should be default table
final_df = pd.merge(df_covid,df_flu,on=['Country','Year','Week'],how='right').sort_values(['Year','Week','Country'])

In [ ]:
final_df.shape

In [ ]:
final_df.head()

In [ ]:
final_df[(final_df.Year==2021) & (final_df.Confirmed.isnull())]

Seems everything is ok so we can change NaN into 0 for covid

In [ ]:
final_df.fillna({'Deaths':0,'Confirmed':0,'Recovered':0},inplace=True)

In [ ]:
final_df.info()

In [ ]:
final_df.describe()

In [ ]:
final_df.Week.unique()

In [ ]:
quarters = pd.DataFrame(columns={"Week","Quarter"})
def quarter(x): 
    if (x <= 13):
        return 1
    elif (x <= 26):
        return 2
    elif (x <= 39):
        return 3
    else:
        return 4

quarters['Week'] = final_df.Week.unique()
quarters['Quarter'] = quarters['Week'].apply(quarter)

In [ ]:
final_df = final_df.merge(quarters,on='Week',how='inner')

In [ ]:
final_df.shape

In [ ]:
final_df.columns

In [ ]:
final_df.Quarter.unique()

In [ ]:
# show me the countries where influenza were detected in this year
final_df[(final_df.Detected_FluCases>0)&(final_df.Year==2021)].sort_values('Detected_FluCases',ascending=False)

In [ ]:
# Here I have compared the number of confirmed covid cases with the covid map (to be sure my data are proper)
final_df[(final_df.Country=='Poland')]['Confirmed'].max()

## Top level table checking and saving the file

In [ ]:
final_df.info()

In [ ]:
final_df.head()

In [ ]:
final_df.describe()

In [ ]:
final_df.to_csv('Covid_and_Influenza.csv',index=False)